# Setup

In [1]:
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, DataCollatorWithPadding, TrainingArguments, Trainer, BertConfig
from datasets import load_dataset
import numpy as np
import evaluate
from torch.utils.data import DataLoader
import math
import torch.nn.modules as nn
from packaging import version
from typing import Optional, Tuple
from torch.nn.attention import SDPBackend, sdpa_kernel
import time
import random
import collections
from transformers import AutoTokenizer

c:\Users\amroa\miniconda3\envs\ML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Custom Attention Class

In [2]:
class CustomAttention(nn.Module):
    def __init__(self, config, num_context_tokens):
        super().__init__()
        # The following equality must be possible num_attention_heads * attention_head_size = config.hidden_size.
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention " f"heads ({config.num_attention_heads})")
        
        # assign attention_head size such that num_attention_heads * attention_head_size = config.hidden_size
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)    #    
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # Initialize projections matrices and dropout layer
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        self.dropout_prob = config.attention_probs_dropout_prob
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)        

        # Central Attention complexity is O(num_context_tokens * sequence_length * hidden_size)
        self.num_context_tokens = num_context_tokens
    
    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size) # (batch_size, num_tokens, num_attention_heads, attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3) # (batch_size, num_attention_heads, num_tokens, attention_head_size)

    # Adapted from BertSelfAttention
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        
        # Project input into query, key, value matrices respectively
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Split q,k, and v for the central attention.
        # Tokens ordering: [CLS] + [CONTEXT_1] + [CONTEXT_2] ... [CONTEXT_N] + ALL SENTENCE TOKENS + [SEP] + [PAD] + [PAD] + [PAD] ....
        # first token is [CLS], then the context tokens, then the sentence, then the [SEP], and finally the padding.
        c = self.num_context_tokens + 1  # [CLS] + context tokens are treated as the central context tokens.
        query_layer_context     = query_layer[:,:,:c,:]
        query_layer_sentence    = query_layer[:,:,c:,:]
        key_layer_context       = key_layer[:,:,:c,:]
        key_layer_sentence      = key_layer[:,:,c:,:]
        value_layer_context     = value_layer[:,:,:c,:]
        value_layer_sentence    = value_layer[:,:,c:,:]

        if attention_mask is not None:
            attention_mask_context  = attention_mask[:,:,:c,c:]
            attention_mask_sentence = attention_mask[:,:,c:,:c]  # should always be zeros, since the central context tokens will never contain padding
            attention_mask_sentence_to_itself = attention_mask.diagonal(dim1=-2,dim2=-1)[:,:,c:].unsqueeze(-1) # (batch_size, 1, n, 1)
          
        if self.num_context_tokens == 0: # Original Attention
            # Take the dot product between "query" and "key" to get the raw attention scores.
            attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2)) / math.sqrt(self.attention_head_size)

            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            if attention_mask is not None:
                attention_scores = attention_scores + attention_mask

            # Normalize the attention scores to probabilities.
            attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)

            # This is actually dropping out entire tokens to attend to, which might
            # seem a bit unusual, but is taken from the original Transformer paper.
            attention_probs = self.dropout(attention_probs)
            attn_output = torch.matmul(attention_probs, value_layer)

        else: # Central Attention 

            attention_scores_context_to_sentence = torch.matmul(query_layer_context, key_layer_sentence.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
            attention_scores_sentence_to_context = torch.matmul(query_layer_sentence, key_layer_context.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
            attention_scores_sentence_to_itself = torch.sum(query_layer_sentence * key_layer_sentence,axis = -1, keepdim=True)/ math.sqrt(self.attention_head_size) # (batch_size, attention_heads, n, 1)

            # print(attention_scores_sentence_to_context.shape) # torch.Size([8, 2, 383, 2])
            # print(attention_scores_sentence_to_itself.shape) # torch.Size([8, 2, 383, 1]) 

            if attention_mask is not None:
                attention_scores_context_to_sentence = attention_scores_context_to_sentence + attention_mask_context
                attention_scores_sentence_to_context = attention_scores_sentence_to_context + attention_mask_sentence
                attention_scores_sentence_to_itself = attention_scores_sentence_to_itself + attention_mask_sentence_to_itself
                # print(attention_mask_sentence_to_itself.shape) # torch.Size([8, 1, 383, 1])
                # print(attention_scores_sentence_to_itself.shape) # torch.Size([8, 2, 383, 1])

            attention_probs_context_to_sentence = torch.nn.functional.softmax(attention_scores_context_to_sentence, dim=-1)
            attention_scores_sentence_to_context_and_itself = torch.cat((attention_scores_sentence_to_context,attention_scores_sentence_to_itself), dim = -1)  # (batch_size, attentino_heads, n, c+1)
            attention_probs_sentence_to_context_and_itself = torch.nn.functional.softmax(attention_scores_sentence_to_context_and_itself, dim=-1)

            # print(attention_scores_sentence_to_context_and_itself.shape) # torch.Size([8, 2, 383, 3])

            attention_probs_context_to_sentence = self.dropout(attention_probs_context_to_sentence)
            attention_probs_sentence_to_context_and_itself = self.dropout(attention_probs_sentence_to_context_and_itself)

            attention_probs_sentence_to_context = attention_probs_sentence_to_context_and_itself[:,:,:,:-1]
            attention_probs_sentence_to_itself = attention_probs_sentence_to_context_and_itself[:,:,:,-1:]

            # print(attention_probs_sentence_to_context.shape) # torch.Size([8, 2, 383, 2])
            # print(attention_probs_sentence_to_itself.shape) # torch.Size([8, 2, 383, 1])

            attn_output_context = torch.matmul(attention_probs_context_to_sentence, value_layer_sentence)
            attn_output_sentence_to_context= torch.matmul(attention_probs_sentence_to_context, value_layer_context)
            attn_output_sentence_to_itself = attention_probs_sentence_to_itself * value_layer_sentence

            # print(attn_output_sentence_to_context.shape) # torch.Size([8, 2, 383, 64])
            # print(attn_output_sentence_to_itself.shape) # torch.Size([8, 2, 383, 64])

            attn_output_sentence = attn_output_sentence_to_context + attn_output_sentence_to_itself

            # (batch_size, num_attention_heads, c, h)

            attn_output = torch.cat((attn_output_context, attn_output_sentence), dim = 2)

        # reformatting attention output to (batch_size, num_tokens, hidden_size), which is the contextual embedding for each token.
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()
        new_attn_output_shape = attn_output.size()[:-2] + (self.all_head_size,)
        attn_output = attn_output.view(new_attn_output_shape)

        # Try those
        # attn_output[:,c:,:] = hidden_states[:,c:,:] 
        # let every token form sentence attend to themselves.

        outputs = (attn_output,)

        return outputs

# Classification Task

## Prepare Dataset Function

In [3]:
def prep_dataset(name, num_context_tokens):

    # Using BERT tokenizer
    checkpoint = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(checkpoint)

    # Add the special context tokens
    special_tokens = {"additional_special_tokens": ["[CONTEXT_"+ str(i) + "]" for i in range(1,num_context_tokens+1)]}
    tokenizer.add_special_tokens(special_tokens)
    context_tokens_ids = [len(tokenizer)-num_context_tokens+i for i in range(num_context_tokens)]

    if name == "sst2":
        raw_dataset = load_dataset("glue", "sst2")
        del raw_dataset["test"] # the labels are -1
        raw_dataset = raw_dataset.remove_columns(["idx"])
        raw_dataset["test"] = raw_dataset["validation"]
        del raw_dataset["validation"]
    elif name == "imdb":
        raw_dataset = load_dataset("imdb")
        del raw_dataset["unsupervised"]
        raw_dataset = raw_dataset.rename_column("text", "sentence")

    def tokenize_and_add_context_tokens(example):    
        output = tokenizer(example["sentence"])
        for i in range(len(output["input_ids"])):
            output["input_ids"][i] = output["input_ids"][i][:1] + context_tokens_ids + output["input_ids"][i][1:]  # [CLS] + context_tokens + sentence
            output["token_type_ids"][i].extend([0] * num_context_tokens)
            output["attention_mask"][i].extend([1] * num_context_tokens)
        return output
    
    tokenized_dataset = raw_dataset.map(tokenize_and_add_context_tokens, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns(["sentence"])
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
    tokenized_dataset.set_format("torch")

    return tokenized_dataset, tokenizer   

## Initialize BERT model with random weights

In [4]:
def initialize_model(num_context_tokens,num_layers=1, max_position_embeddings=4096, intermediate_size=1024, hidden_size=128, num_attention_heads=2):

    vocab_size = 30522
    # Create the model from scratch
    config = BertConfig(
        hidden_size=hidden_size,   # Size of hidden layers
        num_hidden_layers=num_layers,  # Number of transformer layers
        num_attention_heads=num_attention_heads,  # Number of attention heads in each layer
        intermediate_size=intermediate_size,  # Size of the "intermediate" layer (in the feed-forward part of each transformer layer)
        max_position_embeddings= max_position_embeddings,  # Maximum sequence length
    )

    model = BertForSequenceClassification(config)
    model.resize_token_embeddings(vocab_size + num_context_tokens)

    # change the attention of each layer individually 
    for layer in range(num_layers):
        model.bert.encoder.layer[layer].attention.self = CustomAttention(model.config, num_context_tokens)
    
    return model

## Train & Helper Functions

In [6]:
# Prepare training framework
def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2") # just calculates the accuracy
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def train(model, dataset, tokenizer, num_epochs=3, learning_rate = 5e-5, batch_size = 8):

    training_args = TrainingArguments(
        output_dir = "Bert",
        overwrite_output_dir = True,
        run_name = "Bert",
        num_train_epochs = num_epochs,
        learning_rate = learning_rate,
        lr_scheduler_type = "linear",
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        eval_strategy = "no",
        save_strategy = "no",  
        )
    
    trainer = Trainer(
        model,
        training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    return trainer

## Run Experiments  

In [7]:
# if num_context_tokens = 0, then the normal attention will be used, otherwise the central-context attention attention will be used.
def run_experiment(dataset_name,num_context_tokens, seed):
    set_seed(seed)
    dataset, tokenizer = prep_dataset(dataset_name,num_context_tokens)
    model = initialize_model(num_context_tokens=num_context_tokens)
    trainer = train(model,dataset["train"],tokenizer, batch_size = 16)
    acc = trainer.predict(dataset["test"]).metrics["test_accuracy"]

    # log results
    if dataset_name == "sst2":
        with open("SST2_Results.txt", "a") as file:
            file.write("\n")
            file.write(f"Num Context Tokens = {num_context_tokens}      Random Seed = {seed}\n")            
            file.write(f"Accuracy = {acc*100:.2f} %" + "\n")
    else:
        with open("IMDB_Results.txt", "a") as file:
            file.write("\n")
            file.write(f"Num Context Tokens = {num_context_tokens}      Random Seed = {seed}\n")            
            file.write(f"Accuracy = {acc*100:.2f} %" + "\n")

In [8]:
seeds = [2025, 17, 771]
num_context_tokens_options = [0, 1, 8, 32, 128]
datasets = ["sst2", "imdb"]
for dataset_name in datasets:
    for seed in seeds:
        for num_context_tokens in num_context_tokens_options:
            run_experiment(dataset_name, num_context_tokens=num_context_tokens, seed=seed)

C:\Users\amroa\AppData\Local\Temp\ipykernel_8916\176928451.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: amroadil420 (amroadil420-university-of-cambridge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  4%|▍         | 509/12630 [00:09<04:12, 48.00it/s]

{'loss': 0.681, 'grad_norm': 2.402047872543335, 'learning_rate': 4.802058590657166e-05, 'epoch': 0.12}


  8%|▊         | 1006/12630 [00:20<04:15, 45.53it/s]

{'loss': 0.5838, 'grad_norm': 6.353432655334473, 'learning_rate': 4.604117181314331e-05, 'epoch': 0.24}


...

 96%|█████████▌| 4503/4689 [06:26<00:10, 18.52it/s]

{'loss': 0.2586, 'grad_norm': 5.479204177856445, 'learning_rate': 2.015355086372361e-06, 'epoch': 2.88}


100%|██████████| 4689/4689 [06:39<00:00, 11.75it/s]


{'train_runtime': 399.0347, 'train_samples_per_second': 187.954, 'train_steps_per_second': 11.751, 'train_loss': 0.36943519951870607, 'epoch': 3.0}


100%|██████████| 1563/1563 [02:03<00:00, 12.62it/s]


# QA TASK

## Prohibiting the token from attending to itself

In [9]:
class CustomAttention(nn.Module):
    def __init__(self, config, num_context_tokens):
        super().__init__()
        # The following equality must be possible num_attention_heads * attention_head_size = config.hidden_size.
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention " f"heads ({config.num_attention_heads})")
        
        # assign attention_head size such that num_attention_heads * attention_head_size = config.hidden_size
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)    #    
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # Initialize projections matrices and dropout layer
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        self.dropout_prob = config.attention_probs_dropout_prob
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)        

        # Central Attention complexity is O(num_context_tokens * sequence_length * hidden_size)
        self.num_context_tokens = num_context_tokens
    
    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size) # (batch_size, num_tokens, num_attention_heads, attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3) # (batch_size, num_attention_heads, num_tokens, attention_head_size)

    # Adapted from BertSelfAttention
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        
        # Project input into query, key, value matrices respectively
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Split q,k, and v for the central attention.
        # Tokens ordering: [CLS] + [CONTEXT_1] + [CONTEXT_2] ... [CONTEXT_N] + ALL SENTENCE TOKENS + [SEP] + [PAD] + [PAD] + [PAD] ....
        # first token is [CLS], then the context tokens, then the sentence, then the [SEP], and finally the padding.
        c = self.num_context_tokens + 1  # [CLS] + context tokens are treated as the central context tokens.
        query_layer_context     = query_layer[:,:,:c,:]
        query_layer_sentence    = query_layer[:,:,c:,:]
        key_layer_context       = key_layer[:,:,:c,:]
        key_layer_sentence      = key_layer[:,:,c:,:]
        value_layer_context     = value_layer[:,:,:c,:]
        value_layer_sentence    = value_layer[:,:,c:,:]

        if attention_mask is not None:
            attention_mask_context  = attention_mask[:,:,:c,c:]
            attention_mask_sentence = attention_mask[:,:,c:,:c]  # should always be zeros, since the central context tokens will never contain padding
          
        if self.num_context_tokens == 0: # Original Attention
            # Take the dot product between "query" and "key" to get the raw attention scores.
            attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2)) / math.sqrt(self.attention_head_size)

            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            if attention_mask is not None:
                attention_scores = attention_scores + attention_mask

            # Normalize the attention scores to probabilities.
            attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)

            # This is actually dropping out entire tokens to attend to, which might
            # seem a bit unusual, but is taken from the original Transformer paper.
            attention_probs = self.dropout(attention_probs)
            attn_output = torch.matmul(attention_probs, value_layer)

        else: # Central Attention 

            attention_scores_context_to_sentence = torch.matmul(query_layer_context, key_layer_sentence.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
            attention_scores_sentence_to_context = torch.matmul(query_layer_sentence, key_layer_context.transpose(-1, -2)) / math.sqrt(self.attention_head_size)

            if attention_mask is not None:
                attention_scores_context_to_sentence = attention_scores_context_to_sentence + attention_mask_context
                attention_scores_sentence_to_context = attention_scores_sentence_to_context + attention_mask_sentence

            attention_probs_context_to_sentence = torch.nn.functional.softmax(attention_scores_context_to_sentence, dim=-1)
            attention_probs_sentence_to_context = torch.nn.functional.softmax(attention_scores_sentence_to_context, dim=-1)

            attention_probs_context_to_sentence = self.dropout(attention_probs_context_to_sentence)
            attention_probs_sentence_to_context = self.dropout(attention_probs_sentence_to_context)

            attn_output_context = torch.matmul(attention_probs_context_to_sentence, value_layer_sentence)
            attn_output_sentence= torch.matmul(attention_probs_sentence_to_context, value_layer_context)

            attn_output = torch.cat((attn_output_context, attn_output_sentence), dim = 2)

        # reformatting attention output to (batch_size, num_tokens, hidden_size), which is the contextual embedding for each token.
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()
        new_attn_output_shape = attn_output.size()[:-2] + (self.all_head_size,)
        attn_output = attn_output.view(new_attn_output_shape)

        outputs = (attn_output,)

        return outputs

## Dataset Preprocessing

In [10]:
def preprocess_dataset(num_context_tokens, max_length=384, stride=128):

    def preprocess_training_examples(examples):

        questions = [q.strip() for q in examples["question"]]

        inputs = tokenizer(
            questions,
            examples["context"],
            max_length=max_length,
            truncation="only_second",
            stride=stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        for i in range(len(inputs["input_ids"])):
            inputs["input_ids"][i] = inputs["input_ids"][i][:1] + context_tokens_ids + inputs["input_ids"][i][1:]  # [CLS] + context_tokens + sentence
            inputs["token_type_ids"][i] = ([0] * num_context_tokens) + inputs["token_type_ids"][i]
            inputs["attention_mask"][i] = ([1] * num_context_tokens) + inputs["attention_mask"][i]
            inputs["offset_mapping"][i] = inputs["offset_mapping"][i][:1] + ([(0,0)] * num_context_tokens) + inputs["offset_mapping"][i][1:]
            
        offset_mapping = inputs.pop("offset_mapping")
        sample_map = inputs.pop("overflow_to_sample_mapping")
        answers = examples["answers"]
        start_positions = []
        end_positions = []

        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            answer = answers[sample_idx]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = [None] * num_context_tokens + inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

        inputs["start_positions"] = start_positions
        inputs["end_positions"] = end_positions
        return inputs


    def preprocess_validation_examples(examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = tokenizer(
            questions,
            examples["context"],
            max_length=max_length,
            truncation="only_second",
            stride=stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        for i in range(len(inputs["input_ids"])):
            inputs["input_ids"][i] = inputs["input_ids"][i][:1] + context_tokens_ids + inputs["input_ids"][i][1:]  # [CLS] + context_tokens + sentence
            inputs["token_type_ids"][i] = ([0] * num_context_tokens) + inputs["token_type_ids"][i]
            inputs["attention_mask"][i] = ([1] * num_context_tokens) + inputs["attention_mask"][i]
            inputs["offset_mapping"][i] = inputs["offset_mapping"][i][:1] + ([(0,0)] * num_context_tokens) + inputs["offset_mapping"][i][1:]

        sample_map = inputs.pop("overflow_to_sample_mapping")
        example_ids = []

        for i in range(len(inputs["input_ids"])):
            sample_idx = sample_map[i]
            example_ids.append(examples["id"][sample_idx])

            sequence_ids = [None] * num_context_tokens +  inputs.sequence_ids(i)
            offset = inputs["offset_mapping"][i]
            inputs["offset_mapping"][i] = [
                o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
            ]

        inputs["example_id"] = example_ids
        return inputs
    
    raw_datasets = load_dataset("squad")

    # load tokenizer
    model_checkpoint = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 

    # add special tokens
    special_tokens = {"additional_special_tokens": ["[CONTEXT_"+ str(i) + "]" for i in range(1,num_context_tokens+1)]}
    tokenizer.add_special_tokens(special_tokens)
    context_tokens_ids = [len(tokenizer)-num_context_tokens+i for i in range(num_context_tokens)]

    train_dataset = raw_datasets["train"].map(preprocess_training_examples,batched=True,remove_columns=raw_datasets["train"].column_names)
    validation_dataset = raw_datasets["validation"].map(preprocess_validation_examples,batched=True,remove_columns=raw_datasets["validation"].column_names)

    return train_dataset, validation_dataset, tokenizer, raw_datasets

## Evaluation Metric

In [11]:
from tqdm.auto import tqdm
metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

## Initialize Model

In [17]:
def initialize_model(checkpoint, num_context_tokens):
    
    pretrained_model = BertForQuestionAnswering.from_pretrained(checkpoint)
    model = BertForQuestionAnswering(pretrained_model.config)

    for layer in range(pretrained_model.config.num_hidden_layers):
        model.bert.encoder.layer[layer].attention.self = CustomAttention(model.config, num_context_tokens)

    # for p in model.parameters():
    #     print(p[0][:10])
    #     break

    # model.load_state_dict(pretrained_model.state_dict())
    model.resize_token_embeddings(pretrained_model.config.vocab_size + num_context_tokens)

    # for p in model.parameters():
    #     print(p[0][:10])
    #     break

    return model

## Train and Seed functions

In [13]:
def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
def train(model, dataset, tokenizer, batch_size = 8):
    args = TrainingArguments(
        "bert-finetuned-squad",
        evaluation_strategy="no",
        save_strategy="no",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset,
        tokenizer=tokenizer,
    )
    trainer.train()
    return trainer

## Run Experiments  

In [18]:
def run_experiment(num_context_tokens, seed):
# if num_context_tokens = 0, then the normal attention will be used, otherwise the central-context attention attention will be used.
    set_seed(seed)
    train_dataset, validation_dataset, tokenizer, raw_datasets = preprocess_dataset(num_context_tokens)
    model = initialize_model("prajjwal1/bert-tiny", num_context_tokens) # "prajjwal1/bert-small"
    trainer = train(model,train_dataset,tokenizer,batch_size = 8)

    # Evaluation
    predictions, _, _ = trainer.predict(validation_dataset)
    start_logits, end_logits = predictions
    results = compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

    # log results
    with open("From_Scratch_QA_Results.txt", "a") as file:
        file.write("\n")

        file.write(f"Num Context Tokens = {num_context_tokens}      Random Seed = {seed}\n")
        
        # Write the experiment results
        file.write(str(results) + "\n")

    # original_attention_pretrained {'exact_match': 33.40586565752129, 'f1': 45.79680690116114}  , Bert-tiny (25 min)
    # pretrained (bert small) {'exact_match': 45.193945127719964, 'f1': 57.1308813120565}  Bert small (72 min)

In [ ]:
seeds = [2025, 17, 771]
num_context_tokens_options = [0, 1, 8, 32, 128]
for seed in seeds:
    for num_context_tokens in num_context_tokens_options:
        run_experiment(num_context_tokens= num_context_tokens, seed= seed)

# # Num Context Tokens = 0      Random Seed = 2025
# # {'exact_match': 28.94039735099338, 'f1': 40.779770669530336} (20 min)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\amroa\AppData\Local\Temp\ipykernel_8916\4169449671.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  2%|▏         | 501/33198 [00:17<56:53,  9.58it/s]

{'loss': 5.1034, 'grad_norm': 6.323197364807129, 'learning_rate': 1.9698777034761133e-05, 'epoch': 0.05}


  3%|▎         | 1001/33198 [00:54<52:48, 10.16it/s]

{'loss': 4.7121, 'grad_norm': 5.728161811828613, 'learning_rate': 1.939755406952226e-05, 'epoch': 0.09}


  5%|▍         | 1500/33198 [01:24<46:44, 11.30it/s]

...

 99%|█████████▉| 33001/33198 [20:34<00:11, 16.47it/s]

{'loss': 4.0602, 'grad_norm': 9.111811637878418, 'learning_rate': 1.1928429423459245e-07, 'epoch': 2.98}


100%|██████████| 33198/33198 [20:44<00:00, 26.69it/s]


{'train_runtime': 1244.0204, 'train_samples_per_second': 213.479, 'train_steps_per_second': 26.686, 'train_loss': 4.169612178300345, 'epoch': 3.0}


100%|██████████| 10570/10570 [00:21<00:00, 496.32it/s]


# End of Code

In [ ]:
# Reformer	LSH + Reversible layers	O(n log n)	Moderate
# Linformer	Low-rank projection	O(n)	Slight drop
# Performer	Kernel-based attention (FAVOR+)	O(n)	Very good
# BigBird	Structured sparse attention	O(n)	Excellent
# Nyströmformer	Nyström matrix approximation	O(n)	Good
# FlashAttention	CUDA-optimized full attention	O(n²) but fast	No tradeoff

# Logs

Using the 4 layers BERT and 64 batch size:
    the original attention training took 8:17 min and gave an accuracy of 79.82 %
    the proposed attention training took 9:08 and gave an accuracy of 80.96%


It takes 80 min to fine tune Bert-base-uncased on sst2 (batch_size = 128)



QA

Num Context Tokens = 0      Random Seed = 2025
{'exact_match': 28.94039735099338, 'f1': 40.779770669530336}

Num Context Tokens = 1      Random Seed = 2025
{'exact_match': 9.725638599810786, 'f1': 19.227043974258013}

Num Context Tokens = 4      Random Seed = 2025
{'exact_match': 8.684957426679281, 'f1': 18.15225450570832}

Num Context Tokens = 16      Random Seed = 2025
{'exact_match': 9.01608325449385, 'f1': 17.624647379035757}

Num Context Tokens = 32      Random Seed = 2025
{'exact_match': 4.711447492904447, 'f1': 10.245614370758615}

Num Context Tokens = 64      Random Seed = 2025
{'exact_match': 4.654683065279092, 'f1': 10.208962959376224}

Num Context Tokens = 128      Random Seed = 2025
{'exact_match': 3.424787133396405, 'f1': 7.686542942813391}

Num Context Tokens = 0      Random Seed = 17
{'exact_match': 32.02459791863765, 'f1': 44.28220365441659}

Num Context Tokens = 1      Random Seed = 17
{'exact_match': 9.7918637653737, 'f1': 19.70289598515182}

Num Context Tokens = 4      Random Seed = 17
{'exact_match': 7.994323557237465, 'f1': 16.63525060675849}

Num Context Tokens = 16      Random Seed = 17
{'exact_match': 10.189214758751183, 'f1': 19.176374788338425}

Num Context Tokens = 32      Random Seed = 17
{'exact_match': 9.754020813623463, 'f1': 18.620783689403815}

Num Context Tokens = 64      Random Seed = 17
{'exact_match': 7.994323557237465, 'f1': 15.505388032599994}


# Attention (original + mine v1)

In [ ]:
class CustomAttention(nn.Module):
    def __init__(self, config, num_context_tokens):
        super().__init__()
        # The following equality must be possible num_attention_heads * attention_head_size = config.hidden_size.
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention " f"heads ({config.num_attention_heads})")
        
        # assign attention_head size such that num_attention_heads * attention_head_size = config.hidden_size
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)    #    
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # Initialize projections matrices and dropout layer
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        self.dropout_prob = config.attention_probs_dropout_prob
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)        

        # Central Attention complexity is O(num_context_tokens * sequence_length * hidden_size)
        self.num_context_tokens = num_context_tokens
    
    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size) # (batch_size, num_tokens, num_attention_heads, attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3) # (batch_size, num_attention_heads, num_tokens, attention_head_size)

    # Adapted from BertSelfAttention
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        
        # Project input into query, key, value matrices respectively
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Split q,k, and v for the central attention.
        # Tokens ordering: [CLS] + [CONTEXT_1] + [CONTEXT_2] ... [CONTEXT_N] + ALL SENTENCE TOKENS + [SEP] + [PAD] + [PAD] + [PAD] ....
        # first token is [CLS], then the context tokens, then the sentence, then the [SEP], and finally the padding.
        c = self.num_context_tokens + 1  # [CLS] + context tokens are treated as the central context tokens.
        query_layer_context     = query_layer[:,:,:c,:]
        query_layer_sentence    = query_layer[:,:,c:,:]
        key_layer_context       = key_layer[:,:,:c,:]
        key_layer_sentence      = key_layer[:,:,c:,:]
        value_layer_context     = value_layer[:,:,:c,:]
        value_layer_sentence    = value_layer[:,:,c:,:]

        if attention_mask is not None:
            attention_mask_context  = attention_mask[:,:,:c,c:]
            attention_mask_sentence = attention_mask[:,:,c:,:c]  # should always be zeros, since the central context tokens will never contain padding
          
        if self.num_context_tokens == 0: # Original Attention
            # Take the dot product between "query" and "key" to get the raw attention scores.
            attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2)) / math.sqrt(self.attention_head_size)

            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            if attention_mask is not None:
                attention_scores = attention_scores + attention_mask

            # Normalize the attention scores to probabilities.
            attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)

            # This is actually dropping out entire tokens to attend to, which might
            # seem a bit unusual, but is taken from the original Transformer paper.
            attention_probs = self.dropout(attention_probs)
            attn_output = torch.matmul(attention_probs, value_layer)

        else: # Central Attention 

            attention_scores_context_to_sentence = torch.matmul(query_layer_context, key_layer_sentence.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
            attention_scores_sentence_to_context = torch.matmul(query_layer_sentence, key_layer_context.transpose(-1, -2)) / math.sqrt(self.attention_head_size)

            if attention_mask is not None:
                attention_scores_context_to_sentence = attention_scores_context_to_sentence + attention_mask_context
                attention_scores_sentence_to_context = attention_scores_sentence_to_context + attention_mask_sentence

            attention_probs_context_to_sentence = torch.nn.functional.softmax(attention_scores_context_to_sentence, dim=-1)
            attention_probs_sentence_to_context = torch.nn.functional.softmax(attention_scores_sentence_to_context, dim=-1)

            attention_probs_context_to_sentence = self.dropout(attention_probs_context_to_sentence)
            attention_probs_sentence_to_context = self.dropout(attention_probs_sentence_to_context)

            attn_output_context = torch.matmul(attention_probs_context_to_sentence, value_layer_sentence)
            attn_output_sentence= torch.matmul(attention_probs_sentence_to_context, value_layer_context)

            attn_output = torch.cat((attn_output_context, attn_output_sentence), dim = 2)

        # reformatting attention output to (batch_size, num_tokens, hidden_size), which is the contextual embedding for each token.
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()
        new_attn_output_shape = attn_output.size()[:-2] + (self.all_head_size,)
        attn_output = attn_output.view(new_attn_output_shape)

        outputs = (attn_output,)

        return outputs

# Debugging and Testing